<h1>Chess AI: Min Max Algorithm</h1><br>

First off we're using python-chess version 0.15.3 to handle<br>
the board, valid moves, etc. That way we can focus on the AI element.<br>

<b>python-chess: v0.15.3</b>
- https://python-chess.readthedocs.io/en/latest/

In [ ]:
#import the python-chess library
import chess

Next we read from the config.txt file so that we can see how<br>
many moves we're allowed to make before the game ends due to<br>
turn limitation. <br>

In [ ]:
#read in the turn limit
with open('config.txt', 'r') as f:
    turnLim = int(f.readline())
print("The maximum number of moves for a player is: " + str(turnLim))

<h2>1.1 Setting up the Board</h2><br>

The initial state of the board being set up is done as follows:<br>
<font color="red">chess.Board('2n1k3/8/8/8/8/8/8/4K1NR w - - 0 0')</font><br>

The numbers represents empty squares starting from the top left of the board<br>
going line by line from top left to bottom right. Capital letters represent<br>
white pieces and lower case represent black pieces. The w signifies it's white's<br>
turn. 

In [ ]:
def setupBoard(*args):
    if len(args) == 2:
        moveString = args[1].split(' ')[1]
        board = args[0]
        turn = moveString.split(':')[0]
        piece = moveString.split(':')[1]
        move_to = moveString.split(':')[2]
        if turn == "X":
            color = 1 #White
        elif turn == "Y":
            color = 0 #Black
        if piece == "K": 
            piece_type = 6 #King
        if piece == "R":
            piece_type = 4 #Rook
        if piece == "N":
            piece_type = 2  #Knight  
        p = board.pieces(piece_type, color)
        uci = chess.SQUARE_NAMES[list(p)[0]] + move_to
        move = chess.Move.from_uci(uci)
        if move in board.legal_moves:
            board.push(move)
            return True
        else:
            return False
    else:
        return chess.Board('2n1k3/8/8/8/8/8/8/4K1NR w - - 0 0')

Now we can print an ASCII representation of the board.

In [ ]:
b = setupBoard()
print(b)

<h2>1.2 Displaying the Board</h2><br>
Now we need to import two functions for displaying the board.<br>
- display
- SVG

In [ ]:
#Import display for displaying an svg representation of the board
from IPython.display import display, HTML, clear_output, SVG

In [ ]:
#Import time so that we can add a delay before clearing the display
import time

In [ ]:
def showMove(board, player):
    html = ""
    if board == chess.Board('2n1k3/8/8/8/8/8/8/4K1NR w - - 0 0'):
        time.sleep(1)
        html = "<b>Starting Board:</b><br/>%s" % (
           board._repr_svg_())
        clear_output(wait=True)
        display(HTML(html))
        return
    if player == "X":
        file = "log_x.txt"
    elif player == "Y":
        file = "log_y.txt"
    else:
        return
    with open(file, "a") as f:
        move = board.peek()
        piece = str(board.piece_at(move.to_square)).upper()
        if str(board.piece_at(move.to_square)).islower():
            player = "Y" #Black Player is lower case
            color = "Black"
        else:
            player = "X" #White Player is upper case
            color = "White"
        html = "<b>Move %s %s, %s '%s':</b><br/>%s" % (
           len(board.move_stack), color, piece, str(move), board._repr_svg_())
        string = str(len(board.move_stack))+' '+player+':'+piece+':'+ move.uci()[2:]
        f.write(string + '\n')
    #time.sleep(1)
    clear_output(wait=True)
    display(HTML(html))

Now when the initial board is created we can show the Board in a<br>
nice graphical vector image representation!

In [ ]:
b = setupBoard()
showMove(b, player="START")
if setupBoard(b, "1 X:R:h8"):
    showMove(b, player="X")

<h2>1.3 Making a Move</h2>

In [ ]:
def move(board, uci):
    if type(uci) is chess.Move:
        board.push(uci)
        return True
    else:
        move = chess.Move.from_uci(uci)
        if move in board.legal_moves:
            board.push(move)
            return True
    return False

In [ ]:
move(b, "e8d7")
showMove(b, "Y")

<h2>1.4a Dealing with Move Monitoring</h2>

By importing check4Move() we can do a crossplatform<br>
check to see if an opponant has moved by doing a simple<br> 
polling method.

In [ ]:
from filehandling import *

<h2>1.4b Making sure log files are empty</h2>

To make sure that the log_x and log_y text files are empty.<br>
We go ahead and open them up in the following line of code<br>
and zero out the contents.<br>

The function <font color="red">emptyLogFiles()</font> does this.

In [ ]:
emptyLogFiles()

<h2>1.5 Checking for End Game</h2>

Must check to see if the game is finished. If it is<br>
we must return a proper end state string to tell for<br>
what reason it ended.

In [ ]:
def gameOver(board):
    if board.is_stalemate():
        return True
    if board.is_checkmate():
        return True
    return False

def endResult(board, n):
    count = str(len(board.move_stack)) + " half moves or " + str(n) + " full moves"
    if board.is_stalemate():
        return "Stalemate in " + count
    if board.is_checkmate():
        return "Checkmate in " + count

<h2>1.6 Creating the Chess AI</h2>

The functionality for the AI comes from the chessAI file.

In [ ]:
from chessAI import *

<h2>1.7 Playing the Game</h2>

In [ ]:
def play(n, player):   
    board = setupBoard() #Initialize the board
    showMove(board, "START") #Display starting position
    for i in range(1, n+1):
        if gameOver(board):
            return endResult(board, i)
        if player == "X":
            #uci = manualInput(board)
            #uci = randomPlayer(board)
            #uci = stockFish(board, .5)
            uci = computerPlayer(board)
            move(board, uci)
            showMove(board, player)
            if gameOver(board):
                return endResult(board, i)
            s = check4Move(player, len(board.move_stack)+1)
            if setupBoard(board, s):    
                showMove(board, player)
                if gameOver(board):
                    return endResult(board, i)
            else:
                return "Invalid Move made by Player Y: " + s
        if player == "Y":
            s = check4Move(player, len(board.move_stack)+1)
            if setupBoard(board, s):
                showMove(board, player)
                if gameOver(board):
                    return endResult(board, i)
            else:
                return "Invalid Move made by Player X: " + s
            #uci = manualInput(board)
            #uci = randomPlayer(board)
            #uci = stockFish(board, .5)
            uci = computerPlayer(board)
            move(board, uci)
            showMove(board, player)
            if gameOver(board):
                return endResult(board, i)
    return "Maximum # of moves reached"      

In [ ]:
result = play(turnLim, "X")

In [ ]:
print(result)